# Visualization Design

By: *Tyler Biggs*

---

**Overview**

This notebook will go over the design of the vizualizations. It should also serve as a reference for future custom visualizations.

In [1]:
%load_ext autoreload
%autoreload 2
from pprint import pprint

In [2]:
import pandas as pd
import numpy as np
import re
import os
import collections
import itertools
import functools

import bokeh as bk
import bokeh.io
import bokeh.models
import bokeh.layouts
import bokeh.plotting
bokeh.io.output_notebook()

from IPython.display import display, Markdown

# import holoviews as hv
# hv.extension('bokeh')

Loading BokehJS ...

In [3]:
# Path hack to allow imports from the parent directory.
import sys, os
# sys.path.insert(0, os.path.abspath('../'))

In [4]:
# sys.path

In [5]:
from isadream.models import utils
from isadream import io
from isadream.models import nodal, elemental

---

## Dataflow

The data is transfered from the Drupal server as a `.json` file. Those files placed into a directory as the user requests them. That is, all the datasets that a user selects for any given visualziation are placed in a directory. These files are condensed into four dataframes per `.json` file.

In [6]:
# A demo json file is provided.
from isadream import config, demos, demo_path

---

In [38]:
demo_json = io.read_idream_json(os.path.join(demo_path, demos["SIPOS_NMR"]))
node = io.parse_node_json(demo_json)

In [46]:
Markdown(node.as_markdown)

# Sipos 2006, Talanta NMR Figures

**Description**: Extracted figures.

**Submission Date**: *2018-5-25*

**Public Release Date**: *2006-03-13*


---
## Assays
### Potassium Hydroxide Assay
#### Potassium Hydroxide
**('Measurement Condition', '', 'Molar')**: 0.0

**OH-** stoichiometry: 1.0

**K+** stoichiometry: 1.0

**Comment00209**: I manually pulled this points out with a web tool.

### Lithium Hydroxide Assay
#### Lithium Hydroxide
**('Measurement Condition', '', 'Molar')**: 0.0

**OH-** stoichiometry: 1.0

**Li+** stoichiometry: 1.0

### Sodium Hydroxide Assay
#### Sodium Hydroxide
**('Measurement Condition', '', 'Molar')**: 0.0

**Na+** stoichiometry: 1.0

**OH-** stoichiometry: 1.0

## Samples
#### Aluminate Solution
**('Material Property', 'Purity by Weight', 'Percent')**: 0.98

**Al(III)** stoichiometry: 1.0

**('Measurement Condition', '', 'Molar')**: 0.005

**('Material Property', 'Purity by Weight', 'Percent')**: 0.98

**Al(III)** stoichiometry: 1.0

**Al(III)** stoichiometry: 1.0

### Factors
**('Measurement Condition', '', 'Celsius')**: 25.0

**('Measurement', '27_Al', 'ppm')**: 0.0

**('Measurement Condition', '27_Al', 'MHz')**: 78.204

**('Measurement Reference', '[KAl(SO4)2]', '27_Al Reference Compound')**: 0.0

### Comments
**Comment00202**: I manually pulled this points out with a web tool.



In [41]:
# node

In [14]:
node.as_markdown

"## Assays### Potassium Hydroxide Assay\n#### Potassium Hydroxide\n**('Measurement Condition', '', 'Molar')**: 0.0\n\n**OH-** stoichiometry: 1.0\n\n**K+** stoichiometry: 1.0\n\n**Comment00041**: I manually pulled this points out with a web tool.\n\n## Assays### Lithium Hydroxide Assay\n#### Lithium Hydroxide\n**('Measurement Condition', '', 'Molar')**: 0.0\n\n**Li+** stoichiometry: 1.0\n\n**OH-** stoichiometry: 1.0\n\n## Assays### Sodium Hydroxide Assay\n#### Sodium Hydroxide\n**('Measurement Condition', '', 'Molar')**: 0.0\n\n**OH-** stoichiometry: 1.0\n\n**Na+** stoichiometry: 1.0\n\n## Samples#### Aluminate Solution\n**('Material Property', 'Purity by Weight', 'Percent')**: 0.98\n\n**Al(III)** stoichiometry: 1.0\n\n**('Measurement Condition', '', 'Molar')**: 0.005\n\n**('Material Property', 'Purity by Weight', 'Percent')**: 0.98\n\n**Al(III)** stoichiometry: 1.0\n\n**Al(III)** stoichiometry: 1.0\n\n### Factors**('Measurement Condition', '', 'Celsius')**: 25.0\n\n### Factors**('Measu

In [11]:
# for a in node.assays:

In [18]:
for s in node.samples:
#     print(s)
    print(s.as_markdown)
#     for src in s.sources:
#         print(src.as_markdown)

#### Aluminate Solution
**('Material Property', 'Purity by Weight', 'Percent')** stoichiometry: 0.98
**Al(III)** stoichiometry: 1.0
**('Measurement Condition', '', 'Molar')** stoichiometry: 0.005
**('Material Property', 'Purity by Weight', 'Percent')** stoichiometry: 0.98
**Al(III)** stoichiometry: 1.0
**Al(III)** stoichiometry: 1.0



In [13]:
for n in node.factors:
    print(n.as_markdown)

**('Measurement Condition', '', 'Celsius')** stoichiometry: 25.0

**('Measurement', '27_Al', 'ppm')** stoichiometry: 0.0

**('Measurement Condition', '27_Al', 'MHz')** stoichiometry: 78.204

**('Measurement Reference', '[KAl(SO4)2]', '27_Al Reference Compound')** stoichiometry: 0.0



In [15]:
for c in node.comments:
    print(c.as_markdown)

None


In [26]:
node.assays[0].samples[0].get_param_values()

[('comments', []),
 ('factors',
  [Factor(csv_column_index=0, decimal_value=0.0, factor_type='Measurement Condition', name='Factor00098', reference_value='', string_value='', unit_reference='Molar')]),
 ('name', 'SampleNode00101'),
 ('sample_name', 'Potassium Hydroxide'),
 ('sources', []),
 ('species',
  [SpeciesFactor(name='SpeciesFactor00099', species_reference='K+', stoichiometry=1.0),
   SpeciesFactor(name='SpeciesFactor00100', species_reference='OH-', stoichiometry=1.0)])]

In [27]:
# help(node.assays[0].samples[0])

In [28]:
# help(node)

In [29]:
demo_json = io.read_idream_json(os.path.join(demo_path, demos["SIPOS_NMR"]))
node = io.parse_node_json(demo_json)

x_groups=(('Total Aluminate Concentration', ('Molar',), ("Al",)),
          ('Counter Ion Concentration', ('Molar',),
           ("Na+", "Li+", "Cs+", "K+")),
          ('Counter Ion', ('Species',), ("Na+", "Li+", "Cs+", "K+",)),
          ('Base Concentration', ('Molar',), ("OH-",)))

y_groups=(('27 Al ppm', ('ppm',), ("Al",)),)

test_groups = x_groups + y_groups

node_assays = [io.build_node_data(a, test_groups) for a in node.assays]
node_assays
dfs = [pd.DataFrame(cds) for cds, _ in node_assays]
df = pd.concat(dfs)
df

,parent_node,assay_node,Total Aluminate Concentration,sample_node,Counter Ion Concentration,Counter Ion,Base Concentration,27 Al ppm
0,cd6fbd99-4c5e-4292-b523-1e2b4930fd1a,e656262c-6727-419d-b616-bcc4a7c1d07c,0.005,ccd1fedd-c338-489b-acf3-d12ec5892e9b,2.93,K+,2.93,79.90
1,cd6fbd99-4c5e-4292-b523-1e2b4930fd1a,e656262c-6727-419d-b616-bcc4a7c1d07c,0.005,ccd1fedd-c338-489b-acf3-d12ec5892e9b,4.92,K+,4.92,79.84
2,cd6fbd99-4c5e-4292-b523-1e2b4930fd1a,e656262c-6727-419d-b616-bcc4a7c1d07c,0.005,ccd1fedd-c338-489b-acf3-d12ec5892e9b,6.85,K+,6.85,79.72
3,cd6fbd99-4c5e-4292-b523-1e2b4930fd1a,e656262c-6727-419d-b616-bcc4a7c1d07c,0.005,ccd1fedd-c338-489b-acf3-d12ec5892e9b,9.13,K+,9.13,79.66
4,cd6fbd99-4c5e-4292-b523-1e2b4930fd1a,e656262c-6727-419d-b616-bcc4a7c1d07c,0.005,ccd1fedd-c338-489b-acf3-d12ec5892e9b,10.71,K+,10.71,79.66
0,5d040dfc-ce47-4e70-bc4b-3bb894191717,4211fca0-5a83-430c-972a-e947196d7eee,0.005,246172ff-1b3c-43bb-b3b0-3808d843468a,0.66,Li+,0.66,79.92
1,5d040dfc-ce47-4e70-bc4b-3bb894191717,4211fca0-5a83-430c-972a-e947196d7eee,0.005,246172ff-1b3c-43bb-b3b0-3808d843468a,1.10,Li+,1.10,79.84
2,5d040dfc-ce47-4e70-bc4b-3bb894191717,4211fca0-5a83-430c-972a-e947196d7eee,0.005,246172ff-1b3c-43bb-b3b0-3808d843468a,1.64,Li+,1.64,79.66
3,5d040dfc-ce47-4e70-bc4b-3bb894191717,4211fca0-5a83-430c-972a-e947196d7eee,0.005,246172ff-1b3c-43bb-b3b0-3808d843468a,2.14,Li+,2.14,79.54
4,5d040dfc-ce47-4e70-bc4b-3bb894191717,4211fca0-5a83-430c-972a-e947196d7eee,0.005,246172ff-1b3c-43bb-b3b0-3808d843468a,2.59,Li+,2.59,79.36


In [30]:
help(bk.transform.factor_cmap)

Help on function factor_cmap in module bokeh.transform:

factor_cmap(field_name, palette, factors, start=0, end=None, nan_color='gray')
    Create a ``DataSpec`` dict to apply a client-side
    ``CategoricalColorMapper`` transformation to a ``ColumnDataSource``
    column.
    
    Args:
        field_name (str) : a field name to configure ``DataSpec`` with
    
        palette (seq[color]) : a list of colors to use for colormapping
    
        factors (seq) : a sequences of categorical factors corresponding to
            the palette
    
        start (int, optional) : a start slice index to apply when the column
            data has factors with multiple levels. (default: 0)
    
        end (int, optional) : an end slice index to apply when the column
            data has factors with multiple levels. (default: None)
    
        nan_color (color, optional) : a default color to use when mapping data
            from a column does not succeed (default: "gray")
    
    Returns:
   

In [31]:
metadata = [md for _, md in node_assays]
metadata
md = collections.ChainMap(*metadata)
md

ChainMap({'cd6fbd99-4c5e-4292-b523-1e2b4930fd1a': (None, [Comment(body='I manually pulled this points out with a web tool.', comment_title='Study level comment on Sipos 2006.', name='Comment00118')]), 'e656262c-6727-419d-b616-bcc4a7c1d07c': ('Potassium Hydroxide Assay', [Comment(body='I manually pulled this points out with a web tool.', comment_title='Data extraction method.', name='Comment00125')]), 'af1b1837-24ae-42a0-8307-b3fe505597d4': SampleNode(comments=[], factors=[Factor(csv_column_index=None, decimal_value=0.005, factor_type='Measurement Condition', name='Factor00119', reference_value='', string_value='', unit_reference='Molar')], name='SampleNode00124', sample_name='Aluminate Solution', sources=[SourceNode(comments=[], factors=[Factor(csv_column_index=None, decimal_value=0.98, factor_type='Material Property', name='Factor00121', reference_value='Purity by Weight', string_value='', unit_reference='Percent')], name='SourceNode00123', source_name='Aluminum Wire', species=[Specie

---